In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
import onnx

In [ ]:
!pip list

In [ ]:
df = pd.read_csv('relabeled_export_crouch.csv')  # Ersetzen Sie 'data.csv' durch den Pfad Ihrer Datei

# Features und Labels extrahieren
features = df[['x', 'y', 'z', 'yaw', 'pitch', 'roll']].values
labels = df['action'].values

# Labels encodieren
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Feature-Skalierung
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Daten in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(scaled_features, encoded_labels, test_size=0.2, random_state=42)

# Daten in Tensoren umwandeln
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# DataLoader erstellen
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# LSTM-Modell definieren
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        out = self.fc(out)
        return out

In [ ]:
# Hyperparameter
input_size = 6  # Anzahl der Features
hidden_size = 128
output_size = len(label_encoder.classes_)
num_layers = 2
num_epochs = 20
learning_rate = 0.001

# Modell, Verlustfunktion und Optimierer instanziieren
model = LSTMClassifier(input_size, hidden_size, output_size, num_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Training des Modells
model.train()
for epoch in range(num_epochs):
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.unsqueeze(1)  # Batch-Dimension hinzufügen für LSTM
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
# Modell bewerten
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.unsqueeze(1)
        outputs = model(X_batch)
        _, predicted = torch.max(outputs.data, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

    accuracy = correct / total
    print(f'Accuracy: {accuracy * 100:.2f}%')

In [ ]:
# Testen mit neuen Daten
def predict_new_data(new_data, model, scaler, label_encoder):
    # Neue Daten skalieren
    new_data_scaled = scaler.transform(new_data)
    
    # In Tensor umwandeln
    new_data_tensor = torch.tensor(new_data_scaled, dtype=torch.float32).unsqueeze(1)  # Batch-Dimension hinzufügen
    
    # Modell in den Evaluierungsmodus setzen
    model.eval()
    
    with torch.no_grad():
        outputs = model(new_data_tensor)
        _, predicted = torch.max(outputs.data, 1)
    
    # Vorhersagen decodieren
    predicted_labels = label_encoder.inverse_transform(predicted.cpu().numpy())
    
    return predicted_labels

In [ ]:
# Testen mit anderen Daten
"""
-5019.0,23470.0,7634.0,-14.360418,-19.668957,-19.876141,Neutral
-5591.0,27582.0,7211.0,-13.529002,-18.567438,-19.57518,Neutral
-8369.0,26687.0,7139.0,-13.811018,-18.471643,-18.311039,Neutral
-11699.0,22100.0,7059.0,-15.035475,-19.277781,-14.932572,Neutral
-15578.0,17427.0,5996.0,-21.207737,-20.4666,-12.251103,Neutral
-25273.0,10737.0,2789.0,-29.427649,-23.855116,-8.140183,Crouch
-24939.0,4538.0,4197.0,-47.376305,-26.376276,-4.485165,Crouch
-25005.0,12752.0,5266.0,-62.027802,-28.546284,2.15706,Crouch
-26196.0,-15782.0,6594.0,-88.199348,-28.281425,6.477275,Crouch
-27253.0,-22324.0,914.0,-102.161095,-23.648937,16.214033,Crouch
-27887.0,-19877.0,4589.0,-113.087097,-19.671341,22.485985,Crouch
-27773.0,-1554.0,10906.0,-117.005463,-17.334806,22.236414,Crouch
-26835.0,-6510.0,6585.0,-121.647881,-17.750044,16.642509,Crouch
-26077.0,-10775.0,3774.0,-122.92939,-20.976959,7.598702,Crouch
-25294.0,-17870.0,2209.0,-122.441956,-24.102129,7.248987,Crouch
-24993.0,-23676.0,-860.0,-120.279739,-27.141514,10.803078,Crouch
-24331.0,-20957.0,-1861.0,-113.666977,-28.32806,13.204218,Crouch
-23710.0,-15666.0,-3742.0,-107.232986,-30.993572,12.760225,Crouch
-22933.0,-13027.0,-4191.0,-95.226784,-33.565964,9.099196,Crouch
-22730.0,-5367.0,-2946.0,-86.321915,-36.888329,1.072562,Crouch
-22941.0,2860.0,-1892.0,-72.129295,-37.782143,-4.732635,Crouch
-23380.0,11539.0,962.0,-62.240501,-36.857086,-13.824538,Crouch
-20222.0,19113.0,3676.0,-49.183418,-34.959476,-19.523731,Neutral
-15408.0,23036.0,7302.0,-42.288437,-31.399384,-25.70689,Neutral
-13664.0,24529.0,9320.0,-33.258495,-29.620506,-28.231247,Neutral
-9466.0,24071.0,10478.0,-27.782314,-27.430836,-29.805155,Neutral
-7136.0,23920.0,10572.0,-19.93651,-26.271467,-29.415785,Neutral
"""

new_data = [
    [-5019.0,23470.0,7634.0,-14.360418,-19.668957,-19.876141],
    [-5591.0,27582.0,7211.0,-13.529002,-18.567438,-19.57518],
    [-8369.0,26687.0,7139.0,-13.811018,-18.471643,-18.311039],
    [-11699.0,22100.0,7059.0,-15.035475,-19.277781,-14.932572],
    [-15578.0,17427.0,5996.0,-21.207737,-20.4666,-12.251103],
    [-25273.0,10737.0,2789.0,-29.427649,-23.855116,-8.140183],
    [-24939.0,4538.0,4197.0,-47.376305,-26.376276,-4.485165],
    [-25005.0,12752.0,5266.0,-62.027802,-28.546284,2.15706],
    [-26196.0,-15782.0,6594.0,-88.199348,-28.281425,6.477275],
    [-27253.0,-22324.0,914.0,-102.161095,-23.648937,16.214033],
    [-27887.0,-19877.0,4589.0,-113.087097,-19.671341,22.485985],
    [-27773.0,-1554.0,10906.0,-117.005463,-17.334806,22.236414],
    [-26835.0,-6510.0,6585.0,-121.647881,-17.750044,16.642509],
    [-26077.0,-10775.0,3774.0,-122.92939,-20.976959,7.598702],
    [-25294.0,-17870.0,2209.0,-122.441956,-24.102129,7.248987],
    [-24993.0,-23676.0,-860.0,-120.279739,-27.141514,10.803078],
    [-24331.0,-20957.0,-1861.0,-113.666977,-28.32806,13.204218],
    [-23710.0,-15666.0,-3742.0,-107.232986,-30.993572,12.760225],
    [-22933.0,-13027.0,-4191.0,-95.226784,-33.565964,9.099196],
    [-22730.0,-5367.0,-2946.0,-86.321915,-36.888329,1.072562],
    [-22941.0,2860.0,-1892.0,-72.129295,-37.782143,-4.732635],
    [-23380.0,11539.0,962.0,-62.240501,-36.857086,-13.824538],
    [-20222.0,19113.0,3676.0,-49.183418,-34.959476,-19.523731],
    [-15408.0,23036.0,7302.0,-42.288437,-31.399384,-25.70689],
    [-13664.0,24529.0,9320.0,-33.258495,-29.620506,-28.231247],
    [-9466.0,24071.0,10478.0,-27.782314,-27.430836,-29.805155],
    [-7136.0,23920.0,10572.0,-19.93651,-26.271467,-29.415785]
] # Ersetzen Sie dies durch Ihre neuen Daten


predictions = predict_new_data(new_data, model, scaler, label_encoder)
print(predictions)

In [ ]:
# Konvertieren des PyTorch-Modells in ONNX
dummy_input = torch.randn(1, 1, input_size)
torch.onnx.export(
    model, 
    dummy_input, 
    "lstm_model.onnx", 
    input_names=['input'], 
    output_names=['output']
)